Siguiendo https://www.youtube.com/watch?v=kCc8FmEb1nY&t=1535s. 

Status: 9:25, tokenization. 

In [1]:
# Comenzamos descargando el dataset de tiny Shakespeare para una primera implementación.
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-14 10:20:23--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2026-01-14 10:20:23 (16.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# Leer el archivo
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [17]:
print(len(text))

1115394


In [3]:
# Así se ve el dataset
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# En forma cruda
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [5]:
# Cuántos caracteres?
print(len(text))

1115394


In [15]:
# Definimos el vocabulario
# Ordernados de acuerdo al código Unicode
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars)
print("".join(chars))

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [7]:
# Ahora nos preocupamos de la tokenización
# Transformar los caracteres de texto, strings, a una serie de números enteros

# Por simplicidad, usaremos un character-level tokenizer.
# Nota: el tamaño del diccionario y el tamaño de la secuecia de tokens son inversamente proporcionales:
# Puedes tener un diccionarioo vocabulario es pequeño (e.g. ~65 tokens para representar letras, números, y símbolos) y una secuencia de tokens más larga.
# Como ejemplo, el tokenizador te GPT2 tiene ~50k tokens. 


In [11]:
# Creemos un mapeo desde los caracteres a los enteros
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}


In [14]:
# Encoder y decode de strings

def encode(s):
    return [stoi[ch] for ch in s]

def decode(l):
    return ''.join([itos[i] for i in l])

# Ejemplos:

print(encode("hola"))
print(decode([5, 64, 2, 8]))
print(decode(encode("hola")))

[46, 53, 50, 39]
'z!.
hola


In [22]:
# Aplicamos el encoder a todo el texto
import torch

data = torch.tensor(encode(text), dtype=torch.long) # .long sets the int precision to 64 bits, standard in PyTorch
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [23]:
# Hacemos ahora el tr/val split
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [26]:
"""
Ahora, nos disponemos a entrenar el modelo. Nunca entrenamos en todo el texto porque sería
computacionalmente inviable.

Lo que hacemos en realidad es entrenar en pequeños pedazos de texto del training dataset. Estos pedazos tienen un largo máximo,
que se llama block_size.
"""

block_size = 8
print(data[:block_size+1])  


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


In [28]:
"""
En una secuencia de 9 tokens, hay muchos ejemplos de cómo se organizan los tokens.

En una secuencia de 18, 47 probablemente sigue. En una secuencia de 18, 47, 56 probablemente sigue. Y así. Hay varios contextos.

En general, en una secuencia de N tokens, hay N-1 ejemplos que el transformer puede aprender.
"""

# Definimos el contexto
x = train_data[:block_size]
y = train_data[1:block_size+1]

print(x)
print(y)

tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [30]:
# Iteramos sobre tokens
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Cuando el contexto es {context.tolist()} el target es {target.item()}")

# Es útil entrenar sobre contexts de distinta longitud para que el transformer aprenda
# a hacer predicciones incluso con el contexto más corto, context = 1.

Cuando el contexto es [18] el target es 47
Cuando el contexto es [18, 47] el target es 56
Cuando el contexto es [18, 47, 56] el target es 57
Cuando el contexto es [18, 47, 56, 57] el target es 58
Cuando el contexto es [18, 47, 56, 57, 58] el target es 1
Cuando el contexto es [18, 47, 56, 57, 58, 1] el target es 15
Cuando el contexto es [18, 47, 56, 57, 58, 1, 15] el target es 47
Cuando el contexto es [18, 47, 56, 57, 58, 1, 15, 47] el target es 58


In [34]:
# Ahora, lo podemos generalizar
torch.manual_seed(1337)
batch_size = 4 # cuantas secuencias procesamos en paralelo encada forward/backward pass
block_size = 8 # context size

def get_batch(split):
    """
    Generate un pedazo de data con inputs x y output y.
    """

    data = train_data if split == "train" else val_data
    # Genera las posiciones random para extraer el pedazo de texto de ahí.
    # El limite superior de los indices puede ser len(data) - block_size (si no, se saldría del rango)
    # El limite inferior es 0 for default.
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [39]:
xb, yb = get_batch("train")

print("inputs")
print(xb.shape)
print(xb)
print("outputs")
print(yb.shape)
print(yb)

print("----")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"cuando el contexto es {context.tolist()} el target es {target.item()}")

# En este caso, tenemos un set de 32 tokens (independientes, hasta donde el transformer sabe)
# que alimentan al modelo en paralelo.

inputs
torch.Size([4, 8])
tensor([[ 1, 39, 52, 42,  1, 45, 43, 50],
        [ 1, 58, 46, 39, 58,  1, 42, 53],
        [ 1, 61, 53, 59, 50, 42,  1, 21],
        [59, 57, 40, 39, 52, 42,  1, 40]])
outputs
torch.Size([4, 8])
tensor([[39, 52, 42,  1, 45, 43, 50, 42],
        [58, 46, 39, 58,  1, 42, 53,  1],
        [61, 53, 59, 50, 42,  1, 21,  1],
        [57, 40, 39, 52, 42,  1, 40, 47]])
----
cuando el contexto es [1] el target es 39
cuando el contexto es [1, 39] el target es 52
cuando el contexto es [1, 39, 52] el target es 42
cuando el contexto es [1, 39, 52, 42] el target es 1
cuando el contexto es [1, 39, 52, 42, 1] el target es 45
cuando el contexto es [1, 39, 52, 42, 1, 45] el target es 43
cuando el contexto es [1, 39, 52, 42, 1, 45, 43] el target es 50
cuando el contexto es [1, 39, 52, 42, 1, 45, 43, 50] el target es 42
cuando el contexto es [1] el target es 58
cuando el contexto es [1, 58] el target es 46
cuando el contexto es [1, 58, 46] el target es 39
cuando el contexto es [